# CENSUS - ACS 5-year data

In [116]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [117]:
CENSUS_API_KEY = "1d3aeba09035e3cf49f90df444bd1b39b913d5a6"

CENSUS_URL = "https://api.census.gov/data"
CENSUS_YEAR = "2019" # "2019"
CENSUS_DATASET = "acs/acs5"
CENSUS_GEOGRAPHY = "county:*"
URL = f"{CENSUS_URL}/{CENSUS_YEAR}/{CENSUS_DATASET}"

https://api.census.gov/data/2023/acs/acs5?get=NAME,B01001_001E&for=county:*&key=api_key

## One request

In [118]:
CENSUS_VARIABLES = "NAME,B16001_002E"

In [120]:
import requests

# Parámetros de la consulta
params = {
    "get": CENSUS_VARIABLES,  # Nombre del condado y población total
    "for": CENSUS_GEOGRAPHY,          # Obtener datos para todos los condados
    "key": CENSUS_API_KEY              # Tu clave de API
}

# Hacer la petición GET
response = requests.get(URL, params=params)

# Convertir la respuesta en DataFrame
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])  # Primera fila son los nombres de columna
else:
    print("Error en la petición:", response.status_code)
    print(response.text)

Error en la petición: 400
error: invalid 'for' argument


In [103]:
df

,NAME,B16001_002E,state,county
0,"Fayette County, Illinois",None,17,051
1,"Logan County, Illinois",None,17,107
2,"Saline County, Illinois",None,17,165
3,"Lake County, Illinois",None,17,097
4,"Massac County, Illinois",None,17,127
...,...,...,...,...
3215,"Crockett County, Tennessee",None,47,033
3216,"Lake County, Tennessee",None,47,095
3217,"Knox County, Tennessee",None,47,093
3218,"Benton County, Washington",None,53,005


## Create Data

In [104]:
# https://api.census.gov/data/2023/acs/acs5/variables.html
variables = {
    # POPULATION
    f"pop_total_{CENSUS_YEAR}": ["B01001_001E"],
    f"pop_total_male_{CENSUS_YEAR}": ["B01001_002E"],
    f"pop_total_female_{CENSUS_YEAR}": ["B01001_026E"],
    f"pop_18_39_male_{CENSUS_YEAR}": ["B01001_007E", "B01001_008E", "B01001_009E", "B01001_010E", "B01001_011E", "B01001_012E", "B01001_013E"],
    f"pop_18_39_female_{CENSUS_YEAR}": ["B01001_031E", "B01001_032E", "B01001_033E", "B01001_034E", "B01001_035E", "B01001_036E", "B01001_037E"],
    f"pop_40_64_male_{CENSUS_YEAR}": ["B01001_014E", "B01001_015E", "B01001_016E", "B01001_017E", "B01001_018E", "B01001_019E"],
    f"pop_40_64_female_{CENSUS_YEAR}": ["B01001_038E", "B01001_039E", "B01001_040E", "B01001_041E", "B01001_042E", "B01001_043E"],
    f"pop_over_65_male_{CENSUS_YEAR}": ["B01001_020E", "B01001_021E", "B01001_022E", "B01001_023E", "B01001_024E", "B01001_025E"],
    f"pop_over_65_female_{CENSUS_YEAR}": ["B01001_044E", "B01001_045E", "B01001_046E", "B01001_047E", "B01001_048E", "B01001_049E"],
    f"median_age_{CENSUS_YEAR}": ["B01002_001E"],
    f"veterans_{CENSUS_YEAR}": ["B21001_002E"],
    f"inmigrants_{CENSUS_YEAR}": ["B05002_013E"],
    f"labor_force_{CENSUS_YEAR}": ["B23025_003E"],
    
    # RACE
    f"white_{CENSUS_YEAR}": ["B02001_002E"],
    f"black_{CENSUS_YEAR}": ["B02001_003E"],
    f"native_{CENSUS_YEAR}": ["B02001_004E"],
    f"asian_{CENSUS_YEAR}": ["B02001_005E"],
    f"pacific_{CENSUS_YEAR}": ["B02001_006E"],
    f"other_{CENSUS_YEAR}": ["B02001_007E"],
    f"hispanic_{CENSUS_YEAR}": ["B03001_003E"],
    f"two_more_races_{CENSUS_YEAR}": ["B02001_008E"],
    
    # EDUCATION
    f"high_school_{CENSUS_YEAR}": ["B15003_017E"],
    f"bachelors_{CENSUS_YEAR}": ["B15003_022E"],

    # INCOME
    f"median_income_{CENSUS_YEAR}": ["B19013_001E"],
    f"poverty_{CENSUS_YEAR}": ["B17001_002E"],

    # HOUSING
    f"households_median_value_{CENSUS_YEAR}": ["B25077_001E"],
    f"households_avg_size_{CENSUS_YEAR}": ["B25010_001E"],
    f"households_renter_{CENSUS_YEAR}": ["B25003_003E"],
    f"households_owner_{CENSUS_YEAR}": ["B25003_002E"],
    f"households_total_{CENSUS_YEAR}": ["B25003_001E"],
    f"households_median_gross_rent_{CENSUS_YEAR}": ["B25064_001E"],
    f"households_limited_english_{CENSUS_YEAR}": ["C16002_004E", "C16002_007E", "C16002_010E", "C16002_013E", ],
    f"households_no_internet_{CENSUS_YEAR}": ["B28002_013E"],


    # EMPLOYMENT
    f"unemployment_{CENSUS_YEAR}": ["B23025_005E"],
    f"public_transport_{CENSUS_YEAR}": ["B08301_010E"],
    f"mean_travel_time_{CENSUS_YEAR}": ["B08303_001E"],

    # HEALTH
    f"no_health_insurance_{CENSUS_YEAR}": ["B27010_017E", "B27010_033E", "B27010_050E", "B27010_066E"],

}

In [105]:
# Crear DataFrame vacío
df = pd.DataFrame()

# Hacer peticiones una por una
for key, codes in variables.items():
    print(f"Obteniendo datos para: {key}")

    # Realizar la petición GET
    params = {
        "get": ",".join(codes),  # Obtener solo las variables necesarias
        "for": CENSUS_GEOGRAPHY,  # Obtener datos por condado
        "key": CENSUS_API_KEY
    }

    response = requests.get(URL, params=params)

    if response.status_code == 200:
        data = response.json()
        temp_df = pd.DataFrame(data[1:], columns=data[0])  # Primera fila son nombres de columna
        
        # Convertir a numérico
        for col in codes:
            temp_df[col] = pd.to_numeric(temp_df[col], errors="coerce")

        # Si hay más de una variable, sumarlas
        if len(codes) > 1:
            temp_df[key] = temp_df[codes].sum(axis=1)
        else:
            temp_df[key] = temp_df[codes[0]]

        # Eliminar columnas extra
        temp_df = temp_df[["state", "county", key]]

        # Unir con el DataFrame principal
        if df.empty:
            df = temp_df
        else:
            df = df.merge(temp_df, on=["state", "county"], how="outer")

    else:
        print(f"Error al obtener {key}: {response.status_code}")

df

Obteniendo datos para: pop_total_2019
Obteniendo datos para: pop_total_male_2019
Obteniendo datos para: pop_total_female_2019
Obteniendo datos para: pop_18_39_male_2019
Obteniendo datos para: pop_18_39_female_2019
Obteniendo datos para: pop_40_64_male_2019
Obteniendo datos para: pop_40_64_female_2019
Obteniendo datos para: pop_over_65_male_2019
Obteniendo datos para: pop_over_65_female_2019
Obteniendo datos para: median_age_2019
Obteniendo datos para: veterans_2019
Obteniendo datos para: inmigrants_2019
Obteniendo datos para: labor_force_2019
Obteniendo datos para: white_2019
Obteniendo datos para: black_2019
Obteniendo datos para: native_2019
Obteniendo datos para: asian_2019
Obteniendo datos para: pacific_2019
Obteniendo datos para: other_2019
Obteniendo datos para: hispanic_2019
Obteniendo datos para: two_more_races_2019
Obteniendo datos para: high_school_2019
Obteniendo datos para: bachelors_2019
Obteniendo datos para: median_income_2019
Obteniendo datos para: poverty_2019
Obtenien

,state,county,pop_total_2019,pop_total_male_2019,pop_total_female_2019,pop_18_39_male_2019,pop_18_39_female_2019,pop_40_64_male_2019,pop_40_64_female_2019,pop_over_65_male_2019,...,households_renter_2019,households_owner_2019,households_total_2019,households_median_gross_rent_2019,households_limited_english_2019,households_no_internet_2019,unemployment_2019,public_transport_2019,mean_travel_time_2019,no_health_insurance_2019
0,01,001,55380,26934,28446,7904,7875,8666,9447,3594,...,5715,15682,21397,986,153,3683,936,162,23796,3856
1,01,003,212830,103496,109334,26114,26544,34099,37307,19735,...,20034,60896,80930,1020,960,11182,4226,45,87084,18679
2,01,005,25361,13421,11940,4318,2918,4415,3693,2027,...,3654,5691,9345,576,151,3047,849,27,8108,2544
3,01,007,22493,12150,10343,3721,2698,4230,3629,1596,...,1763,5128,6891,734,40,1843,659,0,7910,2201
4,01,009,57681,28495,29186,7375,7131,9684,9776,4609,...,4424,16423,20847,667,369,4983,763,28,21361,6159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,145,52192,24963,27229,7223,7521,7923,9018,4522,...,3804,14917,18721,496,12171,10011,3147,100,14616,4944
3216,72,147,8642,4277,4365,1234,771,1388,1704,866,...,698,1560,2258,627,1592,770,200,0,2306,1040
3217,72,149,22403,10856,11547,3014,3290,3642,3856,1694,...,2415,5493,7908,405,5972,2208,1841,40,6155,836
3218,72,151,33499,16202,17297,4543,4440,5267,6146,3050,...,3235,8306,11541,454,8298,4218,2120,0,8096,1685


### Convert to percentage

In [106]:
# Convertir valores a porcentaje
# **POPULATION**
df[f"pop_total_male_rate_{CENSUS_YEAR}"] = (df[f"pop_total_male_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_total_female_rate_{CENSUS_YEAR}"] = (df[f"pop_total_female_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_18_39_male_rate_{CENSUS_YEAR}"] = (df[f"pop_18_39_male_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_18_39_female_rate_{CENSUS_YEAR}"] = (df[f"pop_18_39_female_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_40_64_male_rate_{CENSUS_YEAR}"] = (df[f"pop_40_64_male_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_40_64_female_rate_{CENSUS_YEAR}"] = (df[f"pop_40_64_female_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_over_65_male_rate_{CENSUS_YEAR}"] = (df[f"pop_over_65_male_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pop_over_65_female_rate_{CENSUS_YEAR}"] = (df[f"pop_over_65_female_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"veterans_rate_{CENSUS_YEAR}"] = (df[f"veterans_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"inmigrants_rate_{CENSUS_YEAR}"] = (df[f"inmigrants_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

# **RACE**
df[f"white_rate_{CENSUS_YEAR}"] = (df[f"white_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"black_rate_{CENSUS_YEAR}"] = (df[f"black_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"native_rate_{CENSUS_YEAR}"] = (df[f"native_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"asian_rate_{CENSUS_YEAR}"] = (df[f"asian_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"pacific_rate_{CENSUS_YEAR}"] = (df[f"pacific_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"other_rate_{CENSUS_YEAR}"] = (df[f"other_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"hispanic_rate_{CENSUS_YEAR}"] = (df[f"hispanic_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"two_more_races_rate_{CENSUS_YEAR}"] = (df[f"two_more_races_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

# **EDUCATION**
df[f"high_school_rate_{CENSUS_YEAR}"] = (df[f"high_school_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100
df[f"bachelors_rate_{CENSUS_YEAR}"] = (df[f"bachelors_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

# **INCOME**
df[f"poverty_rate_{CENSUS_YEAR}"] = (df[f"poverty_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

# **HOUSING**
df[f"households_renter_rate_{CENSUS_YEAR}"] = (df[f"households_renter_{CENSUS_YEAR}"] / df[f"households_total_{CENSUS_YEAR}"]) * 100
df[f"households_owner_rate_{CENSUS_YEAR}"] = (df[f"households_owner_{CENSUS_YEAR}"] / df[f"households_total_{CENSUS_YEAR}"]) * 100
df[f"households_limited_english_rate_{CENSUS_YEAR}"] = (df[f"households_limited_english_{CENSUS_YEAR}"] / df[f"households_total_{CENSUS_YEAR}"]) * 100
df[f"households_no_internet_rate_{CENSUS_YEAR}"] = (df[f"households_no_internet_{CENSUS_YEAR}"] / df[f"households_total_{CENSUS_YEAR}"]) * 100

# **EMPLOYMENT**
df[f"unemployment_rate_{CENSUS_YEAR}"] = (df[f"unemployment_{CENSUS_YEAR}"] / df[f"labor_force_{CENSUS_YEAR}"]) * 100
df[f"public_transport_rate_{CENSUS_YEAR}"] = (df[f"public_transport_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

# **HEALTH**
df[f"no_health_insurance_rate_{CENSUS_YEAR}"] = (df[f"no_health_insurance_{CENSUS_YEAR}"] / df[f"pop_total_{CENSUS_YEAR}"]) * 100

In [107]:
# Redondear todas las columnas que terminan en "_rate_{CENSUS_YEAR}" a 2 decimales
for col in df.columns:
    if col.endswith(f"_rate_{CENSUS_YEAR}"):
        df[col] = df[col].round(2)

### Quitar columnas que ya se han convertido a porcentaje

In [108]:
# Lista de columnas que han sido convertidas a porcentaje
columns_to_remove = [
    f"pop_total_male_{CENSUS_YEAR}",
    f"pop_total_female_{CENSUS_YEAR}",
    f"pop_18_39_male_{CENSUS_YEAR}",
    f"pop_18_39_female_{CENSUS_YEAR}",
    f"pop_40_64_male_{CENSUS_YEAR}",
    f"pop_40_64_female_{CENSUS_YEAR}",
    f"pop_over_65_male_{CENSUS_YEAR}",
    f"pop_over_65_female_{CENSUS_YEAR}",
    f"veterans_{CENSUS_YEAR}",
    f"inmigrants_{CENSUS_YEAR}",
    f"labor_force_{CENSUS_YEAR}",

    # RACE
    f"white_{CENSUS_YEAR}",
    f"black_{CENSUS_YEAR}",
    f"native_{CENSUS_YEAR}",
    f"asian_{CENSUS_YEAR}",
    f"pacific_{CENSUS_YEAR}",
    f"other_{CENSUS_YEAR}",
    f"hispanic_{CENSUS_YEAR}",
    f"two_more_races_{CENSUS_YEAR}",

    # EDUCATION
    f"high_school_{CENSUS_YEAR}",
    f"bachelors_{CENSUS_YEAR}",

    # INCOME
    f"poverty_{CENSUS_YEAR}",

    # HOUSING
    f"households_renter_{CENSUS_YEAR}",
    f"households_owner_{CENSUS_YEAR}",
    f"households_limited_english_{CENSUS_YEAR}",
    f"households_no_internet_{CENSUS_YEAR}",

    # EMPLOYMENT
    f"unemployment_{CENSUS_YEAR}",
    f"public_transport_{CENSUS_YEAR}",

    # HEALTH
    f"no_health_insurance_{CENSUS_YEAR}"
]

# Eliminar las columnas originales
df.drop(columns=columns_to_remove, inplace=True, errors="ignore")

In [109]:
df

,state,county,pop_total_2019,median_age_2019,median_income_2019,households_median_value_2019,households_avg_size_2019,households_total_2019,households_median_gross_rent_2019,mean_travel_time_2019,...,high_school_rate_2019,bachelors_rate_2019,poverty_rate_2019,households_renter_rate_2019,households_owner_rate_2019,households_limited_english_rate_2019,households_no_internet_rate_2019,unemployment_rate_2019,public_transport_rate_2019,no_health_insurance_rate_2019
0,01,001,55380,38.2,58731,154500,2.56,21397,986,23796,...,19.21,10.87,15.06,26.71,73.29,0.72,17.21,3.68,0.29,6.96
1,01,003,212830,43.0,58320,197900,2.59,80930,1020,87084,...,15.90,14.94,10.20,24.75,75.25,1.19,13.82,4.26,0.02,8.78
2,01,005,25361,40.4,32525,90700,2.41,9345,576,8108,...,20.10,5.39,27.11,39.10,60.90,1.62,32.61,9.17,0.11,10.03
3,01,007,22493,40.9,47542,92800,2.99,6891,734,7910,...,25.35,4.64,16.63,25.58,74.42,0.58,26.75,7.28,0.00,9.79
4,01,009,57681,40.7,49358,127800,2.74,20847,667,21361,...,18.57,5.95,13.42,21.22,78.78,1.77,23.90,3.36,0.05,10.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,145,52192,41.5,19617,106600,2.77,18721,496,14616,...,16.99,10.64,45.00,20.32,79.68,65.01,53.47,17.42,0.19,9.47
3216,72,147,8642,47.0,14936,111200,3.83,2258,627,2306,...,26.67,8.01,45.67,30.91,69.09,70.50,34.10,7.50,0.00,12.03
3217,72,149,22403,40.1,19877,92600,2.82,7908,405,6155,...,24.44,11.22,47.50,30.54,69.46,75.52,27.92,21.86,0.18,3.73
3218,72,151,33499,42.9,16295,87200,2.90,11541,454,8096,...,16.76,10.93,53.03,28.03,71.97,71.90,36.55,19.76,0.00,5.03


### Crear County FIPS

In [110]:
df["county_fips"] = df["state"].astype(str) + df["county"].astype(str).str.zfill(3)

### Guardar en CSV

In [111]:
df.to_csv(f"data/census_data_acs_{CENSUS_YEAR}.csv", index=False)
df

,state,county,pop_total_2019,median_age_2019,median_income_2019,households_median_value_2019,households_avg_size_2019,households_total_2019,households_median_gross_rent_2019,mean_travel_time_2019,...,bachelors_rate_2019,poverty_rate_2019,households_renter_rate_2019,households_owner_rate_2019,households_limited_english_rate_2019,households_no_internet_rate_2019,unemployment_rate_2019,public_transport_rate_2019,no_health_insurance_rate_2019,county_fips
0,01,001,55380,38.2,58731,154500,2.56,21397,986,23796,...,10.87,15.06,26.71,73.29,0.72,17.21,3.68,0.29,6.96,01001
1,01,003,212830,43.0,58320,197900,2.59,80930,1020,87084,...,14.94,10.20,24.75,75.25,1.19,13.82,4.26,0.02,8.78,01003
2,01,005,25361,40.4,32525,90700,2.41,9345,576,8108,...,5.39,27.11,39.10,60.90,1.62,32.61,9.17,0.11,10.03,01005
3,01,007,22493,40.9,47542,92800,2.99,6891,734,7910,...,4.64,16.63,25.58,74.42,0.58,26.75,7.28,0.00,9.79,01007
4,01,009,57681,40.7,49358,127800,2.74,20847,667,21361,...,5.95,13.42,21.22,78.78,1.77,23.90,3.36,0.05,10.68,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,145,52192,41.5,19617,106600,2.77,18721,496,14616,...,10.64,45.00,20.32,79.68,65.01,53.47,17.42,0.19,9.47,72145
3216,72,147,8642,47.0,14936,111200,3.83,2258,627,2306,...,8.01,45.67,30.91,69.09,70.50,34.10,7.50,0.00,12.03,72147
3217,72,149,22403,40.1,19877,92600,2.82,7908,405,6155,...,11.22,47.50,30.54,69.46,75.52,27.92,21.86,0.18,3.73,72149
3218,72,151,33499,42.9,16295,87200,2.90,11541,454,8096,...,10.93,53.03,28.03,71.97,71.90,36.55,19.76,0.00,5.03,72151
